In [6]:
# Import packages and libraries
%pip install scikit-learn
%pip install Jinja2
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from pandas import DataFrame
from sklearn.preprocessing import OneHotEncoder

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:

df = pd.read_csv('df_reduced.csv')

----------------------------------------------------------------------

# C4 Data Wrangling
## Reexpression of categorical variables

- Since the dataset contains several categorical variables, it is essential to re-express these variables in a numerical format to include them in the regression analysis. This process is known as one-hot encoding, and it involves creating dummy variables for each category within a categorical variable. The Pandas library provides a method for performing this transformation using the `pd.get_dummies()` function. This function creates a new binary column for each category within a categorical variable, with a value of 1 indicating the presence of that category and 0 indicating its absence. The original categorical variable is then dropped from the dataset to avoid multicollinearity issues in the regression model.

- Ordinal and binary variables will be re-expressed as well using pythons `replace` method.
- Nominal variables will be re-expressed using the `pd.get_dummies()` method in pandas in a method called one-hot encoding.


In [8]:
binary_vars = ['ReAdmis', 'HighBlood', 'Stroke', 'Overweight', 'Arthritis', 'Diabetes', 'Hyperlipidemia', 'BackPain', 'Anxiety', 'Allergic_rhinitis', 'Reflux_esophagitis', 'Asthma']

df[binary_vars] = df[binary_vars].replace({'Yes': 1, 'No': 0})


C:\Users\hinde\AppData\Local\Temp\ipykernel_12636\86739715.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[binary_vars] = df[binary_vars].replace({'Yes': 1, 'No': 0})


In [9]:
# check the unique values for the binary variables
for col in binary_vars:
    print(df[col].value_counts())

ReAdmis
0    6331
1    3669
Name: count, dtype: int64
HighBlood
0    5910
1    4090
Name: count, dtype: int64
Stroke
0    8007
1    1993
Name: count, dtype: int64
Overweight
1    7094
0    2906
Name: count, dtype: int64
Arthritis
0    6426
1    3574
Name: count, dtype: int64
Diabetes
0    7262
1    2738
Name: count, dtype: int64
Hyperlipidemia
0    6628
1    3372
Name: count, dtype: int64
BackPain
0    5886
1    4114
Name: count, dtype: int64
Anxiety
0    6785
1    3215
Name: count, dtype: int64
Allergic_rhinitis
0    6059
1    3941
Name: count, dtype: int64
Reflux_esophagitis
0    5865
1    4135
Name: count, dtype: int64
Asthma
0    7107
1    2893
Name: count, dtype: int64


The data dictionary states: "The (survey) variables represent responses to an eight question survey asking customers to rate the importance of various factors/surfaces on a scale of 1 to 8 (1 most important, 8 least important)"

- The data dictionary states: "The (Survey) variables represent responses to an eight question survey asking customers to rate the importance of various factors/surfaces on a scale of 1 to 8 (1 most important, 8 least important)" Generally 1 is low and 8 is high in terms of importance and having 1 as most important and 8 as least important is not intuitive. Therefore, we will reverse the scale of survey variables so that 1 is the lowest and 8 is the highest in terms of importance. This will make the interpretation more intuitive in the regression analysis.

In [10]:
# reverse the scale of survey variables so that 1 is the lowest and 8 is the highest in terms of importance. df.head is to check before and after the change.
survey_vars = ['S_T_Admission', 'S_T_Treatment', 'S_T_Visits', 'S_Reliability', 'S_Options', 'S_Hours_Treatment', 'S_Staff', 'S_Active_Listening']
df[survey_vars].head()


,S_T_Admission,S_T_Treatment,S_T_Visits,S_Reliability,S_Options,S_Hours_Treatment,S_Staff,S_Active_Listening
0,3,3,2,2,4,3,3,4
1,3,4,3,4,4,4,3,3
2,2,4,4,4,3,4,3,3
3,3,5,5,3,4,5,5,5
4,2,1,3,3,5,3,4,3


In [11]:
# reverse the scale of survey variables so that 1 is the lowest and 8 is the highest in terms of importance. df.head is to check before and after the change.
df[survey_vars] = df[survey_vars].replace({1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1})

df[survey_vars].head()

,S_T_Admission,S_T_Treatment,S_T_Visits,S_Reliability,S_Options,S_Hours_Treatment,S_Staff,S_Active_Listening
0,6,6,7,7,5,6,6,5
1,6,5,6,5,5,5,6,6
2,7,5,5,5,6,5,6,6
3,6,4,4,6,5,4,4,4
4,7,8,6,6,4,6,5,6


In [12]:
# reexpress `complication_risk` from categorical variable to numerical variable where 1 is the lowest and 3 is the highest in terms of risk.
print(df['Complication_risk'].value_counts())
df['Complication_risk'] = df['Complication_risk'].replace({'Low': 1, 'Medium': 2, 'High': 3})
df['Complication_risk'].value_counts()

Complication_risk
Medium    4517
High      3358
Low       2125
Name: count, dtype: int64


C:\Users\hinde\AppData\Local\Temp\ipykernel_12636\2667466588.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Complication_risk'] = df['Complication_risk'].replace({'Low': 1, 'Medium': 2, 'High': 3})


Complication_risk
2    4517
3    3358
1    2125
Name: count, dtype: int64

# ordinal done

In [13]:
#to csv 
df.to_csv('df_for_one_hot.csv', index=False)

In [14]:
#read the csv
df = pd.read_csv('df_for_one_hot.csv')
df

,Area,Children,Age,Income,Marital,Gender,ReAdmis,VitD_levels,Doc_visits,Full_meals_eaten,...,Asthma,S_T_Admission,S_T_Treatment,S_T_Visits,S_Reliability,S_Options,S_Hours_Treatment,S_Staff,S_Active_Listening,Initial_days
0,Suburban,1,53,86575,Divorced,Male,0,19.14,6,0,...,1,6,6,7,7,5,6,6,5,10.59
1,Urban,3,51,46805,Married,Female,0,18.94,4,2,...,0,6,5,6,5,5,5,6,6,15.13
2,Suburban,3,53,14370,Widowed,Female,0,18.06,4,1,...,0,7,5,5,5,6,5,6,6,4.77
3,Suburban,0,78,39741,Married,Male,0,16.58,4,1,...,1,6,4,4,6,5,4,4,4,1.71
4,Rural,1,22,1209,Widowed,Female,0,17.44,5,0,...,0,7,8,6,6,4,6,5,6,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Urban,2,25,45967,Widowed,Male,0,16.98,4,2,...,0,6,7,7,6,5,6,5,7,51.56
9996,Urban,4,87,14983,Widowed,Male,1,18.18,5,0,...,1,6,6,5,7,4,6,5,5,68.67
9997,Rural,3,45,65917,Separated,Female,1,17.13,4,2,...,0,6,6,6,5,5,7,6,7,70.15
9998,Rural,3,43,29702,Divorced,Male,1,19.91,5,2,...,0,4,4,6,5,5,6,5,6,63.36


# One hot encoding


In [17]:

# Using get_dummies to convert nominal variables to 1 and 0 for one-hot encoding
nominal_vars = ['Area', 'Marital', 'Gender', 'Initial_admin']
df_encoded = pd.get_dummies(df, columns=nominal_vars, dtype=int)

# Show the head of the encoded DataFrame
df_encoded.head()


,Children,Age,Income,ReAdmis,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,HighBlood,Stroke,...,Marital_Married,Marital_Never Married,Marital_Separated,Marital_Widowed,Gender_Female,Gender_Male,Gender_Nonbinary,Initial_admin_Elective Admission,Initial_admin_Emergency Admission,Initial_admin_Observation Admission
0,1,53,86575,0,19.14,6,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
1,3,51,46805,0,18.94,4,2,1,1,0,...,1,0,0,0,1,0,0,0,1,0
2,3,53,14370,0,18.06,4,1,0,1,0,...,0,0,0,1,1,0,0,1,0,0
3,0,78,39741,0,16.58,4,1,0,0,1,...,1,0,0,0,0,1,0,1,0,0
4,1,22,1209,0,17.44,5,0,2,0,0,...,0,0,0,1,1,0,0,1,0,0


In [18]:
df_encoded.to_csv('df_encoded.csv', index=False)


In [19]:
df_encoded = pd.read_csv('df_encoded.csv')

Given the considerations about multicollinearity and the distribution of categories within the variables, the columns we chose to drop are:

Area_Rural from the Area variable
Marital_Divorced from the Marital variable
Gender_Female from the Gender variable
Initial_admin_Elective Admission from the Initial_admin variable
These choices align with common practices for handling multicollinearity in datasets with one-hot encoded variables. Dropping these columns helps to avoid the dummy variable trap, a scenario where the independent variables are highly correlated, by ensuring that each set of dummy variables for a categorical feature omits one category. This omission serves as the reference category against which the presence of other categories is measured.

The selection of categories to drop was made to ensure a balanced approach, considering both the need to avoid multicollinearity and the interpretability of the model. While the distribution of categories, such as the fewer instances of "Nonbinary" compared to "Female", might influence the model's learning from underrepresented categories, it does not directly impact the decision to drop a particular category to avoid multicollinearity. The primary goal of dropping one category is to ensure the model's mathematical correctness and interpretability.

In [22]:
# Dropping one level for each one-hot encoded categorical variable based on best practices
columns_to_drop = ['Area_Rural', 'Marital_Divorced', 'Gender_Female', 'Initial_admin_Elective Admission']

# Dropping the selected columns
df_encoded_reduced = df_encoded.drop(columns=columns_to_drop)
# show all columns of head of df_encoded_reduced

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Display all columns of the df_encoded_reduced dataframe
df_encoded_reduced.head()

,Children,Age,Income,ReAdmis,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,HighBlood,Stroke,Complication_risk,Overweight,Arthritis,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,S_T_Admission,S_T_Treatment,S_T_Visits,S_Reliability,S_Options,S_Hours_Treatment,S_Staff,S_Active_Listening,Initial_days,Area_Suburban,Area_Urban,Marital_Married,Marital_Never Married,Marital_Separated,Marital_Widowed,Gender_Male,Gender_Nonbinary,Initial_admin_Emergency Admission,Initial_admin_Observation Admission
0,1,53,86575,0,19.14,6,0,0,1,0,2,0,1,1,0,1,1,1,0,1,6,6,7,7,5,6,6,5,10.59,1,0,0,0,0,0,1,0,1,0
1,3,51,46805,0,18.94,4,2,1,1,0,3,1,0,0,0,0,0,0,1,0,6,5,6,5,5,5,6,6,15.13,0,1,1,0,0,0,0,0,1,0
2,3,53,14370,0,18.06,4,1,0,1,0,2,1,0,1,0,0,0,0,0,0,7,5,5,5,6,5,6,6,4.77,1,0,0,0,0,1,0,0,0,0
3,0,78,39741,0,16.58,4,1,0,0,1,2,0,1,0,0,0,0,0,1,1,6,4,4,6,5,4,4,4,1.71,1,0,1,0,0,0,1,0,0,0
4,1,22,1209,0,17.44,5,0,2,0,0,1,0,0,0,1,0,0,1,0,0,7,8,6,6,4,6,5,6,1.25,0,0,0,0,0,1,0,0,0,0


In [24]:
#FINAL CLEAN TRANSFORMED CSV
df_encoded_reduced.to_csv('medical_transformed.csv', index=False)


--------------------

### G & H: References

- Western Governors University. (2023, December 21). D207 - Medical_clean Dataset. Retrieved from https://lrps.wgu.edu/provision/227079957

- Western Governors University IT Department. (2023). R or Python? How to decide which programming language to learn. Retrieved from https://www.wgu.edu/online-it-degrees/programming-languages/r-or-python.html#

- Datacamp. (2023, December 12). D207 - Exploratory Data Analysis. Retrieved from https://app.datacamp.com/learn/custom-tracks/custom-d207-exploratory-data-analysis 

- Sewell, Dr. (2023). WGU D207 Exploratory Data Analysis [Webinars]. WGU Webex. Accessed December, 2023. https://wgu.webex.com/webappng/sites/wgu/meeting/info/c4aca2eac546482880f1557c938abf40?siteurl=wgu&MTID=me73470c2eac9e863c6f47a3d5b6d2f26 

- Seaborn Developers. (2023). seaborn.scatterplot — seaborn 0.11.2 documentation. Retrieved December 22, 2023, from https://seaborn.pydata.org/generated/seaborn.scatterplot.html

OLD ABOVE _ DELETE?KEEP? as needed.

- Statology. (n.d.). *The Five Assumptions of Multiple Linear Regression*. Statology. Retrieved March 10, 2024, from www.statology.org/multiple-linear-regression-assumptions/

- Pennsylvania State University. (n.d.). *5.3 - The Multiple Linear Regression Model*. STAT 501. Retrieved March 10, 2024, from online.stat.psu.edu/stat501/lesson/5/5.3

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html



In [ ]:
# manage memory by using gc.collect() to clear memory
import gc
gc.collect()


# Limitations

Beware of the following with your regression analysis:

Overfitting can occur due to limited data points.

Multicollinearity occurs when high association (correlation) with other IVs.

P-values can be unreliable and coefficients swing wildly

Check for pairwise correlations and high VIF (> 10)

Tune your model with as many variables as practical. Forward, backward, stepwise
    regression based on AIC, BIC, etc.
ppoint 5 https://westerngovernorsuniversity-my.sharepoint.com/:p:/g/personal/william_sewell_wgu_edu/ERPQ0YpiQktOl-7YyAVnfLMBR5qeBh2cSv61VaJqe_aHKg?e=FjPhPz

# Errata n notes

I'm wrapping up task 1, and my research question is 'what factors influence the total charge a patient receives'. Total charge has a bimodal distribution that I did a log transform on which helped tremendously. Regarding my final reduced model, the RSE is pretty good, both residual normality and homoscedasticity are mostly there. Both have slight variance from expectations around the tails. For fun I decided to re run my code but filtered my data for patients staying less than a month and it improved my RSE, normality and homoscedasticity. Should I change my research question or keep it broad and just explain the limitations of outlier patients?


